# Run Updates

### Connect to Graph

In [12]:
import py2neo
graph = py2neo.Graph(user='neo4j',password='admin')

# Global Root Update

In [13]:
# Count the number of relationships
query_attributenfathers='''
MATCH (n:DunsID)
OPTIONAL MATCH (n)-[r:isChildOf]->(c)
WITH n, count(c) AS nfathers
SET n.nfathers = nfathers
'''
graph.run(query_attributenfathers)

#if there are no relationships then update the isglobalroot attribute
query_attributeglobalroot='''
MATCH (n:DunsID)
WITH n, (CASE n.nfathers WHEN 0 THEN True Else False END) as root
SET n.isglobalroot = root
'''
graph.run(query_attributeglobalroot)

#Update the labels accordingly
query_labelglobalroot='''
MATCH (n:DunsID{isglobalroot:False})
Remove n:GlobalRoot
With count(n) as temp
MATCH (m:DunsID{isglobalroot:True})
Set m:GlobalRoot;
'''
graph.run(query_labelglobalroot)

### Global Root and Direct Parent attribute update for export

#### Duns5

In [14]:
query_duns5='''
MATCH (n:DunsID)-[:isChildOf*0..]->(r:GlobalRoot)
SET n.duns5_calc=r.dunsid;
'''
graph.run(query_duns5)

#### Parent Duns

In [15]:
query_duns3='''
MATCH (n:DunsID{isglobalroot:False})-[:isChildOf]->(r:DunsID)
SET n.parentdunsid=r.dunsid;
'''
graph.run(query_duns3)

query_duns3_bis='''
MATCH (n:GlobalRoot)
SET n.parentdunsid=n.dunsid;'''
graph.run(query_duns3_bis)

# Spent

### Per Duns Node

In [16]:
query_nodespent='''
MATCH(m:DunsID)
SET m.nodespent=0
with count(m) as temp
MATCH (n:DunsID)<-[:hasDuns]-(s:ErpID)
WITH n,SUM(s.spent) as erpspent
SET n.nodespent=erpspent;
'''
a = graph.run(query_nodespent)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 50, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}


In [17]:
### Aggregate per node

In [18]:
old_query='''
MATCH (n:DunsID)<-[:isChildOf*0..]-(child:DunsID)
WITH n,sum(child.nodespent) AS value
SET n.aggregatespent = value
'''

query_leaves='''
MATCH path=(r:GlobalRoot)<-[:isChildOf*0..]-(b:DunsID), (b)<-[:isChildOf]-(l:DunsID)
WITH b, length(path) as depth, collect(l) as ls
ORDER BY depth desc
SET b.aggregatespent = REDUCE(acc = COALESCE(b.nodespent,0), l IN ls | acc + COALESCE(l.aggregatespent, l.nodespent, 0))
'''
a=graph.run(query_leaves)
print(a.stats())

query_root='''
MATCH (n:GlobalRoot)
OPTIONAL MATCH (n)<-[:isChildOf]-(child:DunsID)
WITH n,sum(child.aggregatespent) AS value
SET n.aggregatespent = value+n.nodespent
'''
a=graph.run(query_root)
print(a.stats())

{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 15, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
{'constraints_added': 0, 'constraints_removed': 0, 'indexes_added': 0, 'indexes_removed': 0, 'labels_added': 0, 'labels_removed': 0, 'nodes_created': 0, 'nodes_deleted': 0, 'properties_set': 10, 'relationships_deleted': 0, 'relationships_created': 0, 'contains_updates': True}
